## Data loading

In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
plt.rcParams.update({'text.usetex': True,
                     'text.latex.preamble': r'\usepackage{amsmath}',
                     'font.family': 'serif'})
sns.set(style='ticks', font='serif')

In [ ]:
PATH_RAW_DATA = os.path.join(os.pardir, 'data', 'raw')
PATH_SYN_DATA = os.path.join(os.pardir, 'data', 'processed')
data = pd.read_excel(os.path.join(PATH_RAW_DATA, 'pDeltaT.xlsx'))
clean_data = data.drop(columns='organization')
clean_data.loc[:, 'pDeltaT * 100 [°C]'] = clean_data['pDeltaT [°C]'].values * 100

In [ ]:
clean_data

In [ ]:
clean_data.describe()

In [ ]:
%config InlineBackend.figure_format = 'retina'

cs = sns.color_palette('rocket', 4)
ms = ['o', 'd', 's', '^']
fs = clean_data['f [GHz]'].unique()
ps = ['00', '01', '10', '11']

fig, axs = plt.subplots(2, 2, sharex=False, sharey=True, figsize=(6, 4))
for i, f in enumerate(fs):
    irow, icol = ps[i]
    psPDn_4 = clean_data[clean_data['f [GHz]'] == f]['psPDn_4 [W/m2]']
    psPDtot_4 = clean_data[clean_data['f [GHz]'] == f]['psPDtot_4 [W/m2]']
    psPDn_1 = clean_data[clean_data['f [GHz]'] == f]['psPDn_1 [W/m2]']
    psPDtot_1 = clean_data[clean_data['f [GHz]'] == f]['psPDtot_1 [W/m2]']
    d = clean_data[clean_data['f [GHz]'] == f]['d [mm]']
    axs[int(irow), int(icol)].plot(d, psPDn_4, ms[0], c=cs[0], label=r'$psPD_{\text{n}, 4}$')
    axs[int(irow), int(icol)].plot(d, psPDtot_4, ms[1], c=cs[1], label=r'$psPD_{\text{tot}, 4}$')
    axs[int(irow), int(icol)].plot(d, psPDn_1, ms[2], c=cs[2], label=r'$psPD_{\text{n}, 1}$')
    axs[int(irow), int(icol)].plot(d, psPDtot_1, ms[3], c=cs[3], label=r'$psPD_{\text{tot}, 1}$')
    axs[int(irow), int(icol)].set(xscale='log',
                                  xticks=clean_data['d [mm]'].unique(),
                                  xticklabels=clean_data['d [mm]'].unique(),
                                  yticks=[0, 40, 80],
                                  yticklabels=[0, 40, 80],
                                  ylim=[-5, 85])
    axs[int(irow), int(icol)].set_title(f'$f = {f}$ GHz')
fig.supxlabel('$d$ [mm]')
fig.supylabel(r'incident power density [W/m$^2$]')
fig.suptitle('$\\mathbf{(a)}$', x=0.08, y=0.92)
handles, labels = fig.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(1.25, 0.625))
fig.tight_layout()
sns.despine()

In [ ]:
%config InlineBackend.figure_format = 'retina'

cs = sns.color_palette('rocket', 1)
fs = clean_data['f [GHz]'].unique()
ps = ['00', '01', '10', '11']

fig, axs = plt.subplots(2, 2, sharex=False, sharey=True, figsize=(6, 4))
for i, f in enumerate(fs):
    irow, icol = ps[i]
    pDeltaT = clean_data[clean_data['f [GHz]'] == f]['pDeltaT [°C]']
    d = clean_data[clean_data['f [GHz]'] == f]['d [mm]']
    axs[int(irow), int(icol)].plot(d, pDeltaT, 'o', c=cs[0])                                        
    axs[int(irow), int(icol)].set(xscale='log',
               xticks=clean_data['d [mm]'].unique(),
               xticklabels=clean_data['d [mm]'].unique(),
               yticks=[0, 0.7, 1.4], yticklabels=[0, 0.7, 1.4], ylim=[-0.1, 1.5])
    axs[int(irow), int(icol)].set_title(f'$f = {f}$ GHz')
fig.supxlabel('$d$ [mm]')
fig.supylabel(r'$\Delta T_{max}$ [°C]')
fig.suptitle('$\\mathbf{(b)}$', x=0.08, y=0.92)
fig.tight_layout()
sns.despine()

In [ ]:
%config InlineBackend.figure_format = 'retina'

cs = sns.color_palette('rocket', 4)
ms = ['o', 'd', 's', '^']
As = [4, 4, 1, 1]
fs = clean_data['f [GHz]'].unique()

fig, axs = plt.subplots(2, 2, sharex=False, sharey=True, figsize=(6, 4))
for i, f in enumerate(fs):
    axs[0, 0] = sns.regplot(clean_data[clean_data['f [GHz]'] == f],
                            x='psPDn_4 [W/m2]', y='pDeltaT [°C]', label=f'${f}$',
                            marker=ms[i], color=cs[i], line_kws={'linewidth': 1.5}, ax=axs[0, 0])
    axs[0, 0].set(title=r'$psPD_{\text{n}, 4}$ [W/m$^2$]', xlabel='', ylabel='',
                  xticks=[0, 6, 12], xticklabels=[0, 6, 12], xlim=[-0.5, 12.5],
                  yticks=[0, 0.7, 1.4], yticklabels=[0, 0.7, 1.4], ylim=[-0.1, 1.5])

    axs[0, 1] = sns.regplot(clean_data[clean_data['f [GHz]'] == f],
                            x='psPDtot_4 [W/m2]', y='pDeltaT [°C]', label=f'${f}$',
                            marker=ms[i], color=cs[i], line_kws={'linewidth': 1.5}, ax=axs[0, 1])
    axs[0, 1].set(title=r'$psPD_{\text{tot}, 4}$ [W/m$^2$]', xlabel='', ylabel='',
                  xticks=[0, 16, 32], xticklabels=[0, 16, 32], xlim=[-1, 33],
                  yticks=[0, 0.7, 1.4], yticklabels=[0, 0.7, 1.4], ylim=[-0.1, 1.5])

    axs[1, 0] = sns.regplot(clean_data[clean_data['f [GHz]'] == f],
                            x='psPDn_1 [W/m2]', y='pDeltaT [°C]', label=f'${f}$',
                            marker=ms[i], color=cs[i], line_kws={'linewidth': 1.5}, ax=axs[1, 0])
    axs[1, 0].set(title=r'$psPD_{\text{n}, 1}$ [W/m$^2$]', xlabel='', ylabel='',
                  xticks=[0, 20, 40], xticklabels=[0, 20, 40], xlim=[-2.5, 42.5],
                  yticks=[0, 0.7, 1.4], yticklabels=[0, 0.7, 1.4], ylim=[-0.1, 1.5])

    axs[1, 1] = sns.regplot(clean_data[clean_data['f [GHz]'] == f],
                            x='psPDtot_1 [W/m2]', y='pDeltaT [°C]', label=f'${f}$',
                            marker=ms[i], color=cs[i], line_kws={'linewidth': 1.5}, ax=axs[1, 1])
    axs[1, 1].set(title=r'$psPD_{\text{tot}, 1}$ [W/m$^2$]', xlabel='', ylabel='',
                  xticks=[0, 35, 70], xticklabels=[0, 35, 70], xlim=[-3, 74],
                  yticks=[0, 0.7, 1.4], yticklabels=[0, 0.7, 1.4], ylim=[-0.1, 1.5])
    
fig.supxlabel('incident power density [W/m$^2$]')
fig.supylabel(r'$\Delta T_{max}$ [°C]')
fig.suptitle('$\\mathbf{(a)}$', x=0.08, y=0.92)
handles, labels = fig.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), title='$f$ [GHz]', bbox_to_anchor=(1.15, 0.675))
fig.tight_layout()
sns.despine()

In [ ]:
%config InlineBackend.figure_format = 'retina'

cs = sns.color_palette('rocket', 2)
ms = ['o', 'd', 's', '^']
As = [4, 4, 1, 1]
fs = clean_data['f [GHz]'].unique()
ps = ['00', '01', '10', '11']

fig, axs = plt.subplots(2, 2, sharex=False, sharey=True, figsize=(6, 4))
for i, f in enumerate(fs):
    irow, icol = ps[i]
    if f <= 30:  # A = 4 cm2
        psPDn = clean_data[clean_data['f [GHz]'] == f]['psPDn_4 [W/m2]']
        psPDtot = clean_data[clean_data['f [GHz]'] == f]['psPDtot_4 [W/m2]']
    else:  # A = 1 cm2
        psPDn = clean_data[clean_data['f [GHz]'] == f]['psPDn_1 [W/m2]']
        psPDtot = clean_data[clean_data['f [GHz]'] == f]['psPDtot_1 [W/m2]']
    pDeltaT = clean_data[clean_data['f [GHz]'] == f]['pDeltaT [°C]']
    HFn = pDeltaT / psPDn
    HFtot = pDeltaT / psPDtot
    d = clean_data[clean_data['f [GHz]'] == f]['d [mm]']
    axs[int(irow), int(icol)].plot(d, HFn, ms[0], c=cs[0], label=r'$HF_{\text{n}}$')
    axs[int(irow), int(icol)].plot(d, HFtot, ms[1], c=cs[1], label=r'$HF_{\text{tot}}$')
    axs[int(irow), int(icol)].set(xscale='log',
                                  xticks=clean_data['d [mm]'].unique(),
                                  xticklabels=clean_data['d [mm]'].unique())
    if f <= 30:
        axs[int(irow), int(icol)].set(yticks=[0, 0.05, 0.1],
                                      yticklabels=[0, 0.05, 0.1],
                                      ylim=[-0.01, 0.11])
    axs[int(irow), int(icol)].set_title(f'$f = {f}$ GHz \t $(A = {As[i]}$ cm$^2)$')
fig.supxlabel('$d$ [mm]')
fig.supylabel(r'heating factor [W/m$^2$]')
fig.suptitle('$\\mathbf{(b)}$', x=0.08, y=0.92)
handles, labels = fig.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(1.2, 0.575))
fig.tight_layout()
sns.despine()

## Generating synthetic data

In [ ]:
from sdv.constraints import Inequality, ScalarRange
from sdv.tabular import GaussianCopula, CTGAN
from copulas.univariate import UniformUnivariate, GaussianKDE

In [ ]:
n_tot_rel_1 = Inequality(low_column_name='psPDn_1 [W/m2]',
                         high_column_name='psPDtot_1 [W/m2]')
n_tot_rel_4 = Inequality(low_column_name='psPDn_4 [W/m2]',
                         high_column_name='psPDtot_4 [W/m2]')
table_metadata = {'fields': {'d [mm]': {'type': 'numerical', 'subtype': 'float'},
                             'f [GHz]': {'type': 'numerical', 'subtype': 'float'},
                             'pPDn [W/m2]': {'type': 'numerical', 'subtype': 'float'},
                             'pPDtot [W/m2]': {'type': 'numerical', 'subtype': 'float'},
                             'psPDn_1 [W/m2]': {'type': 'numerical', 'subtype': 'float'},
                             'psPDtot_1 [W/m2]': {'type': 'numerical', 'subtype': 'float'},
                             'psPDn_4 [W/m2]': {'type': 'numerical', 'subtype': 'float'},
                             'psPDtot_4 [W/m2]': {'type': 'numerical', 'subtype': 'float'},
                             'pDeltaT * 100 [°C]': {'type': 'numerical', 'subtype': 'float'}},
                  'constraints': [n_tot_rel_1, n_tot_rel_4]}
field_distributions = {'d [mm]': UniformUnivariate,
                       'f [GHz]': UniformUnivariate,
                       'pPDn [W/m2]': GaussianKDE,
                       'pPDtot [W/m2]': GaussianKDE,
                       'psPDn_1 [W/m2]': GaussianKDE,
                       'psPDtot_1 [W/m2]': GaussianKDE,
                       'psPDn_4 [W/m2]': GaussianKDE,
                       'psPDtot_4 [W/m2]': GaussianKDE,
                       'pDeltaT * 100 [°C]': GaussianKDE}

In [ ]:
copula = GaussianCopula(table_metadata=table_metadata,
                        field_distributions=field_distributions,
                        learn_rounding_scheme=False)
copula.fit(clean_data)
syn_data = copula.sample(num_rows=1000)

## Quality report

In [ ]:
from sdmetrics.reports.single_table import QualityReport

In [ ]:
report = QualityReport()
report.generate(real_data=clean_data.drop(columns='pDeltaT [°C]'),
                synthetic_data=syn_data,
                metadata=table_metadata)

### Column shapes

The similarity of a real column vs. a synthetic column in terms of the column shapes - the marginal distribution of that column - by using the Kolmogorov-Smirnov (KS) test statistics. To compute this statistic, marginal distributions over real and synthetic data are converted into corresponding cumulative distribution functions. The KS statistic quantifies a distance between the empirical distribution functions. Here, the quality score is reported as 1 - KS statistic so that a higher score means higher quality.

In [ ]:
from sdmetrics.reports.utils import get_column_plot

In [ ]:
col_shapes = report.get_details(property_name='Column Shapes')
display(col_shapes)

In [ ]:
get_column_plot(clean_data, syn_data, column_name='psPDn_1 [W/m2]', metadata=table_metadata)

In [ ]:
get_column_plot(clean_data, syn_data, column_name='pDeltaT * 100 [°C]', metadata=table_metadata)

In [ ]:
%config InlineBackend.figure_format = 'retina'

cs = sns.color_palette('rocket', 2)
ps = ['00', '01', '10', '11']
xs = ['psPDn_4 [W/m2]', 'psPDtot_4 [W/m2]', 'psPDn_1 [W/m2]', 'psPDtot_1 [W/m2]']
ls = [r'$psPD_{\text{n}, 4}$', r'$psPD_{\text{tot}, 4}$', r'$psPD_{\text{n}, 1}$', r'$psPD_{\text{tot}, 1}$']

fig, axs = plt.subplots(2, 2, sharex=False, sharey=False, figsize=(6, 4))
for i, x in enumerate(xs):
    irow, icol = ps[i]
    axs[int(irow), int(icol)] = sns.histplot(data=clean_data, x=x,
                                             color=cs[0], ls='-',
                                             stat='density', kde=True,
                                             line_kws={'ls': '-', 'lw': 2,},
                                             label='true',
                                             ax=axs[int(irow), int(icol)])
    axs[int(irow), int(icol)] = sns.histplot(data=syn_data, x=x,
                                             color=cs[1],
                                             stat='density', kde=True,
                                             line_kws={'ls': '--', 'lw': 2,},
                                             label='synthetic',
                                             ax=axs[int(irow), int(icol)])
    axs[int(irow), int(icol)].set(title=ls[i],
                                  xlabel='', ylabel='',
                                 )
fig.supxlabel(r'incident power density [W/m$^2$]')
fig.supylabel(r'probability density')
fig.suptitle('$\\mathbf{(a)}$', x=0.08, y=0.92)
fig.tight_layout()
sns.despine()

In [ ]:
%config InlineBackend.figure_format = 'retina'

cs = sns.color_palette('rocket', 2)

fig, axs = plt.subplots(2, 1, sharex=True, figsize=(6, 4))
axs[0] = sns.histplot(data=clean_data, x='pDeltaT * 100 [°C]',
                      color=cs[0], ls='-', lw=2, stat='density', kde=True, label='true', ax=axs[0])
axs[0] = sns.kdeplot(data=syn_data, x='pDeltaT * 100 [°C]',
                     color=cs[1], ls='--', lw=2, label='synthetic', ax=axs[0])
axs[0].set(title='probability density function', xlabel='', ylabel='',
           xticks=[0, 60, 120], xticklabels=[0, 0.6, 1.2], xlim=[0, 120],
           yticks=[0, 0.015, 0.03], yticklabels=[0, 0.015, 0.03], ylim=[0, 0.03])
axs[1] = sns.kdeplot(data=clean_data, x='pDeltaT * 100 [°C]',
                     color=cs[0], ls='-', lw=2, cumulative=True, label='true', ax=axs[1])
axs[1] = sns.kdeplot(data=syn_data, x='pDeltaT * 100 [°C]',
                     color=cs[1], ls='--', lw=2, cumulative=True, label='synthetic', ax=axs[1])
axs[1].set(title='cumulative density function', xlabel='', ylabel='',
           xticks=[0, 60, 120], xticklabels=[0, 0.6, 1.2], xlim=[0, 120],
           yticks=[0, 0.5, 1], yticklabels=[0, 0.5, 1], ylim=[0, 1])
fig.supxlabel(r'$\Delta T_{max}$ [°C]')
fig.supylabel(r'probability density')
fig.suptitle('$\\mathbf{(b)}$', x=0.08, y=0.92)
handles, labels = fig.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), title='data', bbox_to_anchor=(1.2, 0.615))
fig.tight_layout()
sns.despine()

### Column pair trends

For a pair of columns, the column pair trend is computed as a Pearson correlation coefficient (assuming linear dependence) separateljy on real and synthetic data.
The correlation similarity reported as quality score is computed as the normalized relative difference between two separate correlation coefficients.

In [ ]:
col_pair_trends = report.get_details(property_name='Column Pair Trends')
pDeltaT_pair_trend = col_pair_trends[col_pair_trends['Column 1'] == 'pDeltaT * 100 [°C]']
display(pDeltaT_pair_trend)

In [ ]:
report.get_visualization(property_name='Column Pair Trends')

### Regression tasks

Regression metrics calculate the success of using synthetic data to perform an ML regression task by using linear regression and multi-layer perceptron neural network. It first trains the ML algorithm using the training data (usually synthetic data). The output is an ML model that can predict the value of a given target column. Then, it tests the ML model by making predictions on the testing data (usually real data) and compares against the actual values. It finally returns the Train-Synthetic-Test-Real (TSTR) score by means of the coefficient of determination (R2).

In [ ]:
from sdmetrics.single_table import LinearRegression

In [ ]:
LinearRegression.compute(
    test_data=clean_data.drop(columns='pDeltaT [°C]'),
    train_data=syn_data,
    target='pDeltaT * 100 [°C]',
    metadata=table_metadata
)

## Diagnostic report

Few basic checks on the synthetic data to give a general sense of the strengths and weakness of the synthetic data generation  basic checks on your synthetic data to give a general sense of the strengths and weakness of your synthetic data model.  basic checks on your synthetic data to give a general sense of the strengths and weakness of your synthetic data model.  basic checks on your synthetic data to give a general sense of the strengths and weakness of your synthetic data model. model. 

In [ ]:
from sdmetrics.reports.single_table import DiagnosticReport

In [ ]:
report = DiagnosticReport()
report.generate(real_data=clean_data.drop(columns='pDeltaT [°C]'),
                synthetic_data=syn_data,
                metadata=table_metadata)

In [ ]:
report.get_properties()

In [ ]:
# Does the synthetic data cover the range of possible values?
report.get_details(property_name='Coverage')

In [ ]:
# Is the synthetic data unique or does it copy the real rows?
report.get_details(property_name='Synthesis')

In [ ]:
# Does the synthetic data respect the boundaries set by the real data?
report.get_details(property_name='Boundaries')  # score is the percentage of points within boundaries